In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

from analysis_variables import *

import myPIDselector

import pandas as pd

In [ ]:
start = time.time()

# At Siena
topdir = '/mnt/qnap/babar_data/bnv_plambda'

##josie laptop 
#topdir = "/Users/josieswann/Desktop/important documents"

# On Bellis' laptop
#topdir = '/home/bellis/babar_data/bnv_plambda/'


# At Bellis' home
#topdir = '/home/bellis/babar_data/bnv_plambda'

# On Bellis' laptop
#topdir = './'

#filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Background_and_signal_SP_modes_All_runs.parquet'

data = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")
IS_MC=True


#'''
# Collision data
#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Data_Only_Run_1_BLINDED.parquet'
#filename = f'{topdir}/Data_All_runs_BLINDED.parquet'
#filename = f'{topdir}/Data_All_runs_BLINDED.parquet'

start = time.time()

data_collision = ak.from_parquet(filename)
#data_collision = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")

print(type(data_collision))
#'''


In [ ]:
### information about cross section --> what we'll use to calculate scaling values for histograms 

dataset_information = pd.read_csv("dataset_statistics.csv")
cs_data= pd.read_csv("SP_cross_sections_and_labels.csv")

no_notes= cs_data.drop(["Uncertainty","Note: cross sections found at https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/Cross_sections,_luminosities,_and_other_vital_stats"], axis= 1)
no_notes

In [ ]:
sp = data['spmode']

np.unique(sp.to_list())

In [ ]:
region_definitions

In [ ]:
# Make our histograms
all_hists = bat.create_empty_histograms(hist_defs)

bkg_spmodes = ['998', '1005', '1235', '1237', '3981']
sig_spmodes = ['-999']

spmodes = bkg_spmodes + sig_spmodes

weights = {}
for sp in spmodes:
    weights[sp] = bat.scaling_value(int(sp), dataset_information=dataset_information, cs_data=cs_data, plot=False, verbose=False)
    #weights[sp] = 1

### bat.scaling_value is in Babar_analysis_tools.py 

# Scale the signal higher
weights['-999'] = 1000
weights['0'] = 1

print(weights)
print()
print(spmodes)

In [ ]:
# Pull out some SP mode
spmode = '1005'
#spmode='998'
#spmode='991'
#spmode='-999'
#spmode='0'
#mask_sp = data['spmode'] == spmode

mask_sp = (data['spmode'] == '998') | (data['spmode'] == '1005')


# Collision data
#mask_sp = data['spmode']=='0'

# Make a subset
data_sp = data[mask_sp]

# For data we need to account for blinding
if IS_MC is False:
    nB = data_sp['nB']
    Bp3 = data_sp['Bp3']
    nBp3 = ak.num(Bp3)
    
    mask_to_account_for_blinding = nB == nBp3
    print(len(nB[mask_to_account_for_blinding]), len(nB[~mask_to_account_for_blinding]))
    
    data_sp = data_sp[mask_to_account_for_blinding]

# Use only the data in the fitting region
fit_mask = bat.get_fit_mask(data_sp, region_definitions)

# Make the plot
mes =    ak.flatten(data_sp['BpostFitMes'][fit_mask])
DeltaE = ak.flatten(data_sp['BpostFitDeltaE'][fit_mask])

bat.plot_mes_vs_DeltaE(mes, DeltaE, region_definitions=region_definitions, draw_signal_region=True)

#print(len(mes),len(DeltaE))

## Other HISTOGRAMS 
all_hists = bat.create_empty_histograms(hist_defs)
x = ak.flatten(data_sp['Lambda0_unc_Mass'][fit_mask])
weight = weights[spmode]
all_hists['Lambda0_unc_Mass'].fill(var=x, SP= spmode, cuts= f"{0}", weight= weight)

plt.figure()
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="fill", color='red', label= spmode)
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="step", color='black')
plt.legend();

In [ ]:

mask_lambda0, mask_event_nlambda0 = bat.get_lambda0_mask(data_sp, region_definitions=region_definitions, flightlenvar='Lambda0FlightLen')

# Use only the data in the fitting region
fit_mask = bat.get_fit_mask(data_sp, region_definitions)

######################################


mask_bool_proton, mask_bool_pion, mask_bool_protonB = bat.PID_masks(data_sp, \
              lamp_selector='SuperLooseKMProtonSelection', \
              lampi_selector='VeryTightKMPionMicroSelection', \
              Bp_selector='SuperTightKMProtonSelection', \
              verbosity=0)

mask_pid =      mask_bool_proton & mask_bool_pion & mask_bool_protonB

mask_event = mask_event_nlambda0
mask_candidates = fit_mask & mask_lambda0 & mask_pid 

# Make the plot
mes =    ak.flatten(data_sp[mask_event]['BpostFitMes'][mask_candidates[mask_event]])
DeltaE = ak.flatten(data_sp[mask_event]['BpostFitDeltaE'][mask_candidates[mask_event]])

######################################


bat.plot_mes_vs_DeltaE(mes, DeltaE, region_definitions=region_definitions, draw_signal_region=True, zoom=True)


In [ ]:
#'''
subset = ['BSphr', 'BThrust', 'BCosThetaS', \
          'R2', 'R2All', \
          'thrustMag', 'thrustMagAll', 'thrustCosTh', 'thrustCosThAll', 'sphericityAll', \
          'BCosSphr', 'BCosThetaT', 'BCosThrust', 'BLegendreP2', 'BR2ROE', 'BSphrROE', \
          'BThrustROE']
#'''
#subset = ['BSphr', 'BpostFitDeltaE', 'BpostFitMes', 'BThrust', 'BCosThetaS']

data_for_df = {}

# Pull out some SP mode
spmode = '1005'
#spmode='998'
#spmode='991'
#spmode='-999'
#spmode='0'
#mask_sp = data['spmode'] == spmode

mask_sp = (data['spmode'] == '998') | (data['spmode'] == '1005')


fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(12,12) )

for SIG_OR_BKG in ['bkg', 'sig']:

    mask_sp = None
    if SIG_OR_BKG=='bkg':
        mask_sp = (data['spmode'] == '998') | (data['spmode'] == '1005')
    else:
        mask_sp = (data['spmode'] == '-999')

    print(f"PROCESSING FOR {SIG_OR_BKG} ==========================")
    
    # Collision data
    #mask_sp = data['spmode']=='0'
    
    # Make a subset
    data_sp = data[mask_sp]
    
    # For data we need to account for blinding
    if IS_MC is False:
        nB = data_sp['nB']
        Bp3 = data_sp['Bp3']
        nBp3 = ak.num(Bp3)
        
        mask_to_account_for_blinding = nB == nBp3
        print(len(nB[mask_to_account_for_blinding]), len(nB[~mask_to_account_for_blinding]))
        
        data_sp = data_sp[mask_to_account_for_blinding]
    
    # Use only the data in the fitting region
    fit_mask = bat.get_fit_mask(data_sp, region_definitions)
    
    # Make the plot
    #mes =    ak.flatten(data_sp['BpostFitMes'][fit_mask])
    #DeltaE = ak.flatten(data_sp['BpostFitDeltaE'][fit_mask])
    
    #bat.plot_mes_vs_DeltaE(mes, DeltaE, region_definitions=region_definitions, draw_signal_region=True)
    
    #print(len(mes),len(DeltaE))
    
    ## Other HISTOGRAMS 
    #all_hists = bat.create_empty_histograms(hist_defs)
    #x = ak.flatten(data_sp['Lambda0_unc_Mass'][fit_mask])
    #weight = weights[spmode]
    #all_hists['Lambda0_unc_Mass'].fill(var=x, SP= spmode, cuts= f"{0}", weight= weight)
    
    #plt.figure()
    #all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="fill", color='red', label= spmode)
    #all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="step", color='black')
    #plt.legend();
    
    
    ################# More masks
    mask_lambda0, mask_event_nlambda0 = bat.get_lambda0_mask(data_sp, region_definitions=region_definitions, flightlenvar='Lambda0FlightLen')
    
    # Use only the data in the fitting region
    fit_mask = bat.get_fit_mask(data_sp, region_definitions)
    
    ######################################
    
    
    mask_bool_proton, mask_bool_pion, mask_bool_protonB = bat.PID_masks(data_sp, \
                  lamp_selector='SuperLooseKMProtonSelection', \
                  lampi_selector='VeryTightKMPionMicroSelection', \
                  Bp_selector='SuperTightKMProtonSelection', \
                  verbosity=0)
    
    mask_pid =      mask_bool_proton & mask_bool_pion & mask_bool_protonB
    
    #mask_event = mask_event_nlambda0
    mask_candidates = fit_mask & mask_lambda0 & mask_pid 

    npass = ak.num(mask_candidates[mask_candidates])
    
    mask_event = npass==1
    
    # Make the plot
    #mes =    ak.flatten(data_sp[mask_event]['BpostFitMes'][mask_candidates[mask_event]])
    #DeltaE = ak.flatten(data_sp[mask_event]['BpostFitDeltaE'][mask_candidates[mask_event]])
    
    ######################################
    
    
    #bat.plot_mes_vs_DeltaE(mes, DeltaE, region_definitions=region_definitions, draw_signal_region=True, zoom=True)
    
    
    #plt.figure(figsize=(12,12))
    for i,var in enumerate(subset):
        #print(var)
    
        if var[0]=='B':
            x = data_sp[mask_event][var][mask_candidates[mask_event]]
            x = ak.flatten(x)
        else:
            x = data_sp[mask_event][var]
    
        #plt.subplot(5,4,i+1)
        row = int(i/5)
        col = i%5
        #print(row, col)
        plt.sca(axes[row][col])
        alpha = 1.0
        if SIG_OR_BKG == 'sig':
            alpha=0.5

        plt.hist(x,bins=40, label=f'{SIG_OR_BKG}: {var}', density=True, alpha=alpha);

        if SIG_OR_BKG == 'sig':
            plt.legend()

        # Put it in dictionary
        if SIG_OR_BKG == 'bkg':
            data_for_df[var] = x.to_list()
            if i==0:
                data_for_df['dataset'] = np.full(len(x), SIG_OR_BKG).tolist()
        elif SIG_OR_BKG == 'sig':
            data_for_df[var] += x.to_list()
            if i==0:
                data_for_df['dataset'] += np.full(len(x), SIG_OR_BKG).tolist()


In [ ]:
#data_for_df
len(data_for_df['dataset'])

In [ ]:
for key,val in data_for_df.items():
    print(key,len(val))

In [ ]:
df = pd.DataFrame.from_dict(data_for_df)

df

In [ ]:
#fig, axes = plt.subplots(nrows=4, ncols=5) 

In [ ]:
#axes[2]

In [ ]:
mask_candidates = fit_mask & mask_lambda0 & mask_pid 

mask_candidates[mask_candidates]

In [ ]:
ak.num(mask_candidates[mask_candidates])